# Pytorch Basics

## 0. Table of Contents

#### 1. Basic autograd example 1
#### 2. Basic autograd example 2
#### 3. Loading data from Numpy
#### 4. Input pipeline
#### 5. Input pipeline for custom dataset
#### 6. Pretrained model
#### 7. Save and load model

In [47]:
# import some related libraries
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader, TensorDataset

## 1. Basic autograd example 1

In [48]:
# Create tensors
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [49]:
# Build a computational graph
y = w * x + b     # y = 2 * x + 3

In [50]:
# Compute gradients
y.backward()

In [51]:
# Print out the gradients
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


## 2. Basic autograd example 2

In [52]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

In [53]:
# Build a fully connected layer
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('w.shape = ', linear.weight.shape)
print('b: ', linear.bias)
print('b.shape = ', linear.bias.shape)

w:  Parameter containing:
tensor([[-0.1703,  0.0372,  0.0361],
        [-0.5494, -0.4684, -0.3580]], requires_grad=True)
w.shape =  torch.Size([2, 3])
b:  Parameter containing:
tensor([0.3386, 0.3091], requires_grad=True)
b.shape =  torch.Size([2])


In [54]:
# Build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [55]:
# Forward pass
pred = linear(x)

In [56]:
# Compute loss
loss = criterion(pred, y)
print("loss: ", loss.item())

loss:  1.8267806768417358


In [57]:
# Print out the gradients
print('dL/dw: ', linear.weight.grad)
print('dL/db: ', linear.bias.grad)

dL/dw:  None
dL/db:  None


In [58]:
# 1-step gradient descent
optimizer.step()

In [59]:
# Print out the loss after 1-step gradient descent
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  1.8267806768417358


## 3. Loading data from Numpy

In [60]:
# Create a numpy array
x = np.array([[1, 2], [3, 4]])
x

array([[1, 2],
       [3, 4]])

In [61]:
# Convert the numpy array to a torch tensor
y = torch.from_numpy(x)
y

tensor([[1, 2],
        [3, 4]])

In [73]:
# Convert the torch tensor to a numpy array
z = y.numpy()
z

array([[1, 2],
       [3, 4]])

## 4. Input pipeline

In [77]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../data',
                                                                train=True,
                                                                transform=transforms.ToTensor(),
                                                                download=True)

Files already downloaded and verified


In [92]:
# Data loader (this provides queues and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset='../data/cifar-10-python.tar.gz',
                                                           batch_size=64,
                                                           shuffle=True)

In [93]:
train_loader

In [94]:
# Fectch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)
print(len(train_dataset))

torch.Size([3, 32, 32])
6
50000


In [95]:
# When iteration starts, queue and thread start to load data from files
data_iter = iter(train_loader)

In [96]:
# Mini-batch images and labels
# images, labels = data_iter.next()

In [103]:
# CIFAR 10 official site
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [111]:
cifar10_dataset = unpickle("../data/cifar-10-batches-py/data_batch_1")
print(type(cifar10_dataset))
cifar10_dataset.keys()

<class 'dict'>


dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

## 5. Input pipeline for custom dataset

In [114]:
# You should build your custom dataset as below
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names
        pass
    
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using Numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data
        # 3. Return a data pair (image and label)
        pass
    
    def __len__(self):
        # You should change 0 to the total size of your dataset
        return 0

In [113]:
# You can then use the prebuilt data loader
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                                            batch_size=64,
                                                            shuffle=True)

ValueError: num_samples should be a positive integer value, but got num_samples=0

## 6. Pretrained model 

In [ ]:
#. Download and load the pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

In [ ]:
# If you want to fintune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
# Replace the top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100)    # 100 is an example

In [ ]:
# Forward pass
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.size())       # (64, 100)

## 7. Save and load the model

In [ ]:
# Save and load the entire model
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

In [ ]:
# Save and load only the model parameters (recommended)
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))